In [ ]:
import requests
import json
from operator import itemgetter

from openai import OpenAI

azure_subscription_key = 'your_own_bing_key'
openai_api_key = 'your_own_openai_key'
model = "gpt-3.5-turbo"

In [3]:
def PreferredStories(newsiteURL, profile):
    # Construct a requrest to the Bing News Search API
    search_url = "https://api.bing.microsoft.com/v7.0/news/search"
    headers = {"Ocp-Apim-Subscription-Key" : azure_subscription_key}
    params  = {"q": f"site:{newsiteURL}", "count": 20}

    try:
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
    except Exception as ex:
        raise ex

    articles = [[value['name'], value['description'], value['url']] for value in search_results['value']]

    # OpenAI API
    client = OpenAI(api_key=openai_api_key)
    messages = [
        {"role": 'system', "content": "You are a helpful assistant."},
        {"role": 'user', "content": f"Given the profile {profile}, just provide the relevant score (0-10) to each of the news articles in pure json format: [\u007b\"score\": score\u007d, ...] {articles}"},
    ]
    response = client.chat.completions.create(model = model, messages = messages)
    ai_summary = json.loads(response.choices[0].message.content)
    articles = [article + [ai_summary[idx]['score']] for idx, article in enumerate(articles)]

    # Sort articles by relevance and select top 5
    articles = sorted(articles, key=itemgetter(3), reverse=True)[:5]

    # Print the top 5 articles
    for article in articles:
        print("<title> " + f"{article[0]}" + " </title>")
        print("<summary> " + f"{article[1]}" + " </summary>")
        print("<link> " + f"{article[2]}" + " </link>")
        print("<score> " + f"{article[3]}" + " </score>")
        print("\n")

In [5]:
# Test the function
profile = {
    "Name": "Yanting Chen",
    "Job Role": "Software Engineer",
    "Sector": "Wireless Technology",
}
PreferredStories("techcrunch.com", profile)

<title> Amazon reverses course, revokes police access to Ring footage via Neighbors app </title>
<summary> Amazon today announced that it is end-of-lifing Request for Assistance (RFA), a controversial tool that allowed police and fire departments to request doorbell video through Ring’s Neighbors app. “Public safety agencies like fire and police departments ... </summary>
<link> https://techcrunch.com/2024/01/24/amazon-reverses-course-revokes-police-access-to-ring-footage-via-neighbors-app/ </link>
<score> 9 </score>


<title> DXwand raises $4M to scale its conversational AI platform for enterprises in MENA </title>
<summary> DXwand, a Cairo- and Dubai-based startup that leverages conversational AI to help businesses in the Middle East automate customer service and employee assistance, has raised $4 million in a Series A round. UAE-based Shorooq Partners and Cairo-headquartered ... </summary>
<link> https://techcrunch.com/2024/01/25/dxwand-raises-4m-to-scale-its-conversational-ai-platf